## Stacking Transfer Learning for Pneumothorax Classification in Keras
Detect Pneumothorax with 0.81 Accuracy

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True 
session = InteractiveSession(config=config)

### Load Packages and Data

In [ ]:
import os
import cv2 
import sys 
import random
import warnings
import numpy as np
import pandas as pd  
from time import time 
import matplotlib.pyplot as plt 
from tqdm import tqdm 
from itertools import chain 
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label 

import tensorflow as tf 
from tensorflow.keras.layers import Dense, Input, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate, Activation, Add, multiply, add, concatenate, LeakyReLU, ZeroPadding2D, UpSampling2D, BatchNormalization, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K 

from sklearn.metrics import classification_report, confusion_matrix
%matplotlib inline

In [ ]:
PATH = '../input/pneumothorax-chest-xray-images-and-masks/siim-acr-pneumothorax/'
IMG_PATH = PATH + 'png_images/'
IMG_DATA = PATH + 'stage_1_train_images.csv'
IMG_TEST_DATA = PATH + 'stage_1_test_images.csv'

train_df = pd.read_csv(IMG_DATA)
test_df = pd.read_csv(IMG_TEST_DATA)

### Random Over-Sampling

In [ ]:
train_df_0 = train_df[train_df['has_pneumo'] == 0]
train_df_1 = train_df[train_df['has_pneumo'] == 1]

train_1_over = train_df_1.sample(len(train_df_0), replace=True)
train_df_oversampling = pd.concat([train_df_0, train_1_over], axis=0)

print("Random Over-Sampling")
print(train_df_oversampling.has_pneumo.value_counts())

In [ ]:
train_df_oversampling['has_pneumo'] = train_df['has_pneumo'].astype(str)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, 
                             validation_split=0.25, 
                             zoom_range=0.1,
                             rotation_range=0.2,
                            #  height_shift_range=0.2,
                            #  width_shift_range=0.2,
                             fill_mode='nearest')

train_generator = datagen.flow_from_dataframe(
    dataframe=train_df_oversampling,
    directory=IMG_PATH,
    x_col='new_filename',
    y_col='has_pneumo',
    subset='training',
    batch_size=32,
    shuffle=True,
    class_mode='categorical',
    target_size=(128, 128)
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=train_df_oversampling,
    directory=IMG_PATH,
    x_col='new_filename',
    y_col='has_pneumo',
    subset='validation',
    batch_size=32,
    shuffle=True, 
    class_mode='categorical',
    target_size=(128, 128)
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=IMG_PATH,
    x_col='new_filename',
    y_col=None,
    batch_size=32,
    shuffle=False,
    class_mode=None,
    target_size=(128, 128)
)

### "Ensemble" Transfer Learning

In [ ]:
!pip install git+https://github.com/qubvel/efficientnet

In [ ]:
import efficientnet.keras as efn

In [ ]:
from tensorflow.keras.applications import DenseNet121, ResNet50V2, DenseNet169, InceptionResNetV2,MobileNetV2, NASNetMobile, DenseNet201, NASNetLarge, Xception
from tensorflow.keras.layers import Average


def nasnet(model_input):
    base_model = NASNetMobile(weights='imagenet', include_top=True, input_tensor=model_input)
    last = base_model.output 
    x = Flatten()(last)
    x = Dense(1024)(x)
    x = LeakyReLU()(x)
    x = Dropout(0.5)(x)
    x = Dense(512)(x)
    x = LeakyReLU()(x)
    x = Dropout(0.3)(x)
    x = Dense(128)(x)
    x = LeakyReLU()(x)
    x = Dropout(0.2)(x)
    predictions = Dense(2, activation="sigmoid")(x)
    model = Model(inputs= base_model.input, outputs= predictions)
    
#     for layer in model.layers:
#         layer._name = layer.name + str("_2")
    
    return model 

def mobilenet(model_input):
    base_model = MobileNetV2(weights='imagenet', include_top=True, input_tensor=model_input)
    last = base_model.output 
    x = Flatten()(last)
    x = Dense(1024)(x)
    x = LeakyReLU()(x)
    x = Dropout(0.5)(x)
    x = Dense(512)(x)
    x = LeakyReLU()(x)
    x = Dropout(0.3)(x)
    x = Dense(128)(x)
    x = LeakyReLU()(x)
    x = Dropout(0.2)(x)
    predictions = Dense(2, activation="sigmoid")(x)
    model = Model(inputs= base_model.input, outputs= predictions)

    for layer in model.layers:
        layer._name = layer.name + str("_2")

    return model 

model_input = Input(shape=(128, 128, 3))
nasnet_model = nasnet(model_input)
mobilenet_model = mobilenet(model_input)
ensemble_model = [nasnet_model, mobilenet_model]
def ensemble(models, model_input):
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    model = Model(model_input, y, name='ensemble')
    return model

In [ ]:
ensembleNN = ensemble(ensemble_model, model_input=model_input)

In [ ]:
ensembleNN.summary()

In [ ]:
# from keras.utils.vis_utils import plot_model
# plot_model(ensembleNN, to_file='modelplot.png', show_shapes=True, show_layer_names=True)

In [ ]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

reduce_learning_rate = ReduceLROnPlateau(
    monitor='val_loss', factor=0.25, patience=2, verbose=1, mode='auto',
    min_delta=1e-10, cooldown=0, min_lr=0
)

early_stopping = EarlyStopping(
    monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto',
    baseline=None, restore_best_weights=True
)

callbacks = [reduce_learning_rate, early_stopping]

In [ ]:
ensembleNN.compile(Adam(lr=0.0001, decay=1e-6),loss='binary_crossentropy',metrics=['accuracy'])

### Train

In [ ]:
ensembleNN.fit_generator(train_generator, epochs=50, validation_data=valid_generator, callbacks=callbacks)

### Evaluate

In [ ]:
ensembleNN.evaluate_generator(generator=valid_generator, verbose=1)

### Predict

In [ ]:
y_pred = ensembleNN.predict_generator(test_generator, verbose=1)

In [ ]:
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
y_true = np.asanyarray(test_df['has_pneumo'])

In [ ]:
print(classification_report(y_pred, y_true))